In [1]:
import glob
import pandas as pd
import numpy as np
import pickle
import re
from bs4 import BeautifulSoup
import collections
import requests
from itertools import chain
from tqdm import tqdm
import string

In [2]:
# init index list
tree_links_index = []
# Extract index pages
for i in range(1, 8):
    tree_links_index.append('http://www.llifle.com/Encyclopedia/TREES/Species/all/{0}/100/'.format(i))

# Init empty list
tree_links = []

for index_pages in tqdm(tree_links_index):
    # Extract XML
    URL = index_pages
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, 'html.parser')
    # Extract links incomplete
    tree_links_half = soup.find_all('a')

    # Complete the links
    tree_links_temp = ['http://www.llifle.com' + pages.get('href') for pages in tree_links_half
                           if pages.get('href') != None 
                           if pages.get('href').startswith('/Encyclopedia/TREES/Family/')]
    # Add to all trees
    tree_links += tree_links_temp

100%|█████████████████████████████████████████████| 7/7 [00:04<00:00,  1.72it/s]


In [5]:
with open('../../data/processed/intermediate_web_llifle_treelist.pkl', 'wb') as f:
    pickle.dump(tree_links, f)

In [ ]:
# Init empty dict
data = collections.defaultdict(list)

for URL in tqdm(tree_links):

    # Get Page
    page = requests.get(URL)
    # Structure page
    soup = BeautifulSoup(page.content, 'html.parser')
    # Name
    name = soup.title.text.replace('\n', '')
    # Loop over text
    for text in soup.find_all('p'):
        try:
            if text.find('b').text.strip().startswith('Description'):
                data[name].append(tuple([1, text.text]))
            else:
                data[name].append(tuple([0, text.text]))
        except:
            continue

In [ ]:
with open('../../data/processed/descriptions_web_trees_llifle.pkl', 'wb') as f:
    pickle.dump(data, f)